In [1]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy
import dill

In [2]:
## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
## Additional rotors used by Kriegsmarine M3 and M4 only:
VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

## reflectors, standard army & airforce
contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

notches = {'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}

Notches
Rotor    Notch 	Window 	next left rotor steps when rotor steps from/to

I        Y      Q       Q -> R
II       M      E       E -> F
III      D      V       V -> W
IV       R      J       J -> K
V        H      Z       Z -> A
VI-VIII  H+U    Z, M    Z -> A and M -> N

Rotor 	Notch 	Window 	next left rotor steps when rotor steps from/to

I
II
III
IV
V
VI VII VIII 

	

Y
M
D
R
H
H + U

	

Q
E
V
J
Z
Z and M

	

Q -> R
E -> F
V -> W
J -> K
Z -> A
Z -> A and M -> N

In [3]:
raw_rotors = {'I':I, 'II':II, 'III':III, 'IV':IV, 'V':V}
print(raw_rotors)

{'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}


In [18]:
forward_rotors = {'I':[], 'II':[], 'III':[], 'IV':[], 'V':[]}
for k in forward_rotors.keys():
    forward_rotors[k] = [ascii_uppercase.index(c) for c in raw_rotors[k]]
# rI = [ascii_uppercase.index(c) for c in I]
for k,v in forward_rotors.items():
    print(k,':',v)

I : [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9]
II : [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4]
III : [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14]
IV : [4, 18, 14, 21, 15, 25, 9, 0, 24, 16, 20, 8, 17, 7, 23, 11, 13, 5, 19, 6, 10, 3, 2, 12, 22, 1]
V : [21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23, 0, 22, 12, 9, 16, 14, 5, 4, 2, 10]


In [5]:
rev_rotors = {}

for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

for k,v in rev_rotors.items():
    print(k,':',v)

I : [20, 22, 24, 6, 0, 3, 5, 15, 21, 25, 1, 4, 2, 10, 12, 19, 7, 23, 18, 11, 17, 8, 13, 16, 14, 9]
II : [0, 9, 15, 2, 25, 22, 17, 11, 5, 1, 3, 10, 14, 19, 24, 20, 16, 6, 4, 13, 7, 23, 12, 8, 21, 18]
III : [19, 0, 6, 1, 15, 2, 18, 3, 16, 4, 20, 5, 21, 13, 25, 7, 24, 8, 23, 9, 22, 11, 17, 10, 14, 12]
IV : [7, 25, 22, 21, 0, 17, 19, 13, 11, 6, 20, 15, 23, 16, 2, 4, 9, 12, 1, 18, 10, 3, 24, 14, 8, 5]
V : [16, 2, 24, 11, 23, 22, 4, 13, 5, 19, 25, 14, 18, 12, 21, 9, 20, 3, 10, 6, 8, 0, 17, 15, 7, 1]


In [6]:
reflectors = {'B': {k:v for k,v in zip(entry, breflector)}, 'C': {k:v for k,v in zip(entry, creflector)}}
print(reflectors)

{'B': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'C': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}


In [19]:
class Enigma3:
    
    def __init__(self,left_rotor,middle_rotor,right_rotor,reflector,menu_link='ZZZ'):
        """rotors must be strings referring to either ['I','II','III','IV','V']
        reflector must be string, one of either ['B','C']"""
        
        self.right_rotor = right_rotor
        self.middle_rotor = middle_rotor
        self.left_rotor = left_rotor
        self.reflector = reflectors[reflector]
        self.menu_link = menu_link
        self.middle_notch = entry.index(notches[self.middle_rotor])   ## point if right rotor reaches will trigger middle rotor to step
        self.left_notch = entry.index(notches[self.left_rotor])  ## point if middle rotor reaches will trigger left rotor to step
        self.pos_left_rotor, self.pos_mid_rotor, self.pos_rgt_rotor = (ascii_uppercase.index(m) for m in menu_link.upper())
        self.in_status = {char:0 for char in entry}
        self.out_status = {char:0 for char in entry}

        
    def once_thru_scramble(self,start_character, direction, first_rotor, pos1, second_rotor, pos2, 
                       third_rotor, pos3):
        """ start_character must be single ASCII character A-Z
        direction is either 'forward' or 'back' """
        if direction == 'forward':
            usedict = {k:v for k,v in forward_rotors.items()}
        elif direction == 'back':
            usedict = {k:v for k,v in rev_rotors.items()}
        else:
            print('only forward or back for direction')
            return 'wtf'

        start_character = start_character.upper()
        entry_pos = entry.index(start_character)
        fst_pos_modifier = (26 + pos1 - 0)%26
        fst_in = (entry_pos + fst_pos_modifier)%26
        fst_out = usedict[first_rotor][fst_in]
        ch1o = entry[fst_out]

        scd_pos_modifier = (26 + pos2 - pos1)%26
        scd_in = (fst_out + scd_pos_modifier)%26
        ch2i = entry[scd_in]
        scd_out = usedict[second_rotor][scd_in]
        ch2o = entry[scd_out]

        thd_pos_modifier = (26 + pos3 - pos2)%26
        thd_in = (scd_out + thd_pos_modifier)%26
        ch3i = entry[thd_in]
        thd_out = usedict[third_rotor][thd_in]
        ch3o = entry[thd_out]

#         if direction == 'forward':
#             print(f"{start_character} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
#         elif direction == 'back':
#             print(f"{start_character} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")

        return ch3o
    
    def full_scramble(self,in_ch):
        in_ch = in_ch.upper()
        left_rotor = self.left_rotor
        middle_rotor = self.middle_rotor
        right_rotor = self.right_rotor
        rflector = self.reflector
        ## first run right to left through scrambler
        forward_run = self.once_thru_scramble(in_ch, direction='forward', first_rotor=right_rotor, pos1=self.pos_rgt_rotor, 
                                              second_rotor=middle_rotor, pos2=self.pos_mid_rotor, 
                                              third_rotor=left_rotor, pos3=self.pos_left_rotor)

        ## reflector back around for return
        rfi_pos_mod = (26 + 0 - self.pos_left_rotor)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
        rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
        chri = entry[rf_in]
        mirrored = rflector[chri]

#         print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")

        ## second run back left to right thru scrambler
        back_run = self.once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=self.pos_left_rotor, 
                                      second_rotor=middle_rotor, pos2=self.pos_mid_rotor, third_rotor=right_rotor, pos3=self.pos_rgt_rotor)

        bk_out = entry.index(back_run)
        bko_pos_mod = (26 + 0 - self.pos_rgt_rotor)%26  ## as above, '0' just reflects that the entry interface doesn't move
        bk_final = (bk_out + bko_pos_mod)%26
        final = entry[bk_final]
#         print('RR back out:  ', back_run, '-->', final)
#         print(in_ch,"-->",final)
        return final
    
    def rotor_step(self,rotor_position):
        """"""
        if rotor_position == 25:
            rotor_position = 0
        else:
            rotor_position += 1
        return rotor_position
    
    def step_enigma(self):
        """Just acts on itself, steps the rotors"""
        
#         print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")
        
        if self.pos_rgt_rotor == self.middle_notch and self.pos_mid_rotor == self.left_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            self.pos_left_rotor = self.rotor_step(self.pos_left_rotor)
            print('--- left & middle rotor step')
        elif self.pos_rgt_rotor == self.middle_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            print('--- middle rotor step ---')
        else:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            
        print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")

        
    def only_ascii(self,instring):
        """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
        newstring = ''
        for character in instring:
            if character in ascii_letters:
                newstring += character.upper()
        return newstring

    def enigmatise(self,tocode, startset='AAA'):
        encoded = ''

        tocode = self.only_ascii(tocode)
        
        self.pos_left_rotor, self.pos_mid_rotor, self.pos_rgt_rotor = (ascii_uppercase.index(s) for s in startset.upper())
        
        for c in tocode:
            print('in:    ', c)
            self.step_enigma()

            print('rp = ', self.pos_rgt_rotor, 'mp = ', self.pos_mid_rotor, 'lp = ', self.pos_left_rotor)
            
            out = self.full_scramble(c)
            print('out:   ', out,'\n')
            
            encoded += out

        return encoded
            


In [20]:
eg1 = Enigma3('I','II','III','B')

In [163]:
def once_thru_scramble(ch, direction='forward', first_rotor='I', pos1=0, second_rotor='II', pos2=0, 
                       third_rotor='III', pos3=0):
    """ ch must be single ASCII character A-Z
    direction is either 'forward' or 'back' """
    if direction == 'forward':
        usedict = {k:v for k,v in base_rotors.items()}
    elif direction == 'back':
        usedict = {k:v for k,v in rev_rotors.items()}
    else:
        print('only forward or back for direction')
        return 'wtf'
    
    entry = ascii_uppercase
    ch = ch.upper()
    entry_pos = entry.index(ch)
    fst_pos_mod = (26 + pos1 - 0)%26
#     print(fst_pos_mod)
    fst_in = (entry_pos + fst_pos_mod)%26
#     print(fst_in)
    fst_out = usedict[first_rotor][fst_in]
#     print(fst_out)
    ch1o = entry[fst_out]
#     print('first rotor:  ', ch, '-->', ch1o)
    
    scd_pos_mod = (26 + pos2 - pos1)%26
    scd_in = (fst_out + scd_pos_mod)%26
    ch2i = entry[scd_in]
#     print('r1 to r2:     ', ch1o, '-->', ch2i)
#     print(scd_pos_mod, scd_in)
    scd_out = usedict[second_rotor][scd_in]
#     print(scd_out)
    ch2o = entry[scd_out]
#     print('second rotor: ', ch2i, '-->', ch2o)
    
    thd_pos_mod = (26 + pos3 - pos2)%26
    thd_in = (scd_out + thd_pos_mod)%26
    ch3i = entry[thd_in]
#     print('r2 to r3:     ', ch2o, '-->', ch3i)
    thd_out = usedict[third_rotor][thd_in]
#     print(thd_out)
    ch3o = entry[thd_out]
#     print('third rotor:  ', ch3i, '-->', ch3o)
    
    
    if direction == 'forward':
        print(f"{ch} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
    elif direction == 'back':
        print(f"{ch} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")
    
    
    return ch3o
    
    
once_thru_scramble('q')

Q -> (RR out) X -> (MR in) X -> (MR out) V -> (LR in) V -> (LR out) M


'M'

In [164]:
def full_scramble(in_ch, left_rotor='III', lpos=0, middle_rotor='II', mpos=0, right_rotor='I', rpos=0, rflector='Brf'):
    entry = ascii_uppercase
    in_ch = in_ch.upper()
    
    ## first run right to left through scrambler
    forward_run = once_thru_scramble(in_ch, 'forward', first_rotor=right_rotor, pos1=rpos, second_rotor=middle_rotor,
                                     pos2=mpos, third_rotor=left_rotor, pos3=lpos)
    
    ## reflector back around for return
    rfi_pos_mod = (26 + 0 - lpos)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
    rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
    chri = entry[rf_in]
    mirrored = reflectors[rflector][chri]
    
    print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")
    
    ## second run back left to right thru scrambler
    back_run = once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=lpos, 
                                  second_rotor=middle_rotor, pos2=mpos, third_rotor=right_rotor, pos3=rpos)
    
    bk_out = entry.index(back_run)
    bko_pos_mod = (26 + 0 - rpos)%26  ## as above, '0' just reflects that the entry interface doesn't move
    bk_final = (bk_out + bko_pos_mod)%26
    final = entry[bk_final]
    print('RR back out:  ', back_run, '-->', final)
    
    return final

# full_scramble('i')

In [165]:
full_scramble('q', lpos=2, mpos=22, rpos=10)

Q -> (RR out) E -> (MR in) Q -> (MR out) Q -> (LR in) W -> (LR out) U
U -> S (into reflector) -> F (reflected out)
F -> (LR out) D -> (MR in) X -> (MR out) I -> (RR in) W -> (RR out) N
RR back out:   N --> D


'D'

In [9]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [167]:
def step_rotor(p_in):
    """"""
    if p_in == 25:
        p_in = 0
    else:
        p_in += 1
    return p_in

In [168]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [169]:
only_ascii(word)

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [170]:
sht = 'pat'
sht = sht.upper()

lt, mt, rt = (ascii_uppercase.index(s) for s in sht)
print(lt, mt, rt)

15 0 19


In [172]:
def enigmatise(tocode, startset='AAA', lstartpos=0, mstartpos=0, rstartpos=0):
    encoded = ''

    tocode = only_ascii(tocode)
    
    lftpos, midpos, rgtpos = (ascii_uppercase.index(s) for s in startset.upper())
    
    
    # start out initial positions as starting positions
#     lftpos = lstartpos
#     midpos = mstartpos
#     rgtpos = rstartpos

    ## TO DO need to enable notches to be defined by ring setting as input to function
    mnotch = 4   ## point if right rotor reaches will trigger middle rotor to step
    lnotch = 5  ## point if middle rotor reaches will trigger left rotor to step

    for c in tocode:
        print('in:    ', c)

        ## simple stepping version - notches step M & L straight away
#         rgtpos = step_rotor(rgtpos)
#         if rgtpos == mnotch:
#             midpos = step_rotor(midpos)
#             print('--- middle rotor step ---')
#             if midpos == lnotch:
#                 lftpos = step_rotor(lftpos)
#                 print('---  left rotor step  ---')
                
        rgtpos = step_rotor(rgtpos)
        
        if rgtpos == mnotch:
            
            midpos = step_rotor(midpos)
            print('--- middle rotor step ---')
            if midpos == lnotch:
                lftpos = step_rotor(lftpos)
                print('---  left rotor step  ---')

        print('rp = ', rgtpos, 'mp = ', midpos, 'lp = ', lftpos)
        out = full_scramble(c, lpos=lftpos, mpos=midpos, rpos=rgtpos)
        print('out:   ', out,'\n')
        encoded += out

    return encoded

In [10]:
opcode = 'IND'
## operator set starting position, traditionally different for each new message

In [21]:
eg1.enigmatise(word,opcode)

in:     O
Lpos=8, Mpos=13, Rpos=4
rp =  4 mp =  13 lp =  8
out:    N 

in:     H
--- middle rotor step ---
Lpos=8, Mpos=14, Rpos=5
rp =  5 mp =  14 lp =  8
out:    K 

in:     W
Lpos=8, Mpos=14, Rpos=6
rp =  6 mp =  14 lp =  8
out:    N 

in:     H
Lpos=8, Mpos=14, Rpos=7
rp =  7 mp =  14 lp =  8
out:    M 

in:     A
Lpos=8, Mpos=14, Rpos=8
rp =  8 mp =  14 lp =  8
out:    W 

in:     T
Lpos=8, Mpos=14, Rpos=9
rp =  9 mp =  14 lp =  8
out:    G 

in:     A
Lpos=8, Mpos=14, Rpos=10
rp =  10 mp =  14 lp =  8
out:    E 

in:     B
Lpos=8, Mpos=14, Rpos=11
rp =  11 mp =  14 lp =  8
out:    D 

in:     E
Lpos=8, Mpos=14, Rpos=12
rp =  12 mp =  14 lp =  8
out:    R 

in:     A
Lpos=8, Mpos=14, Rpos=13
rp =  13 mp =  14 lp =  8
out:    L 

in:     U
Lpos=8, Mpos=14, Rpos=14
rp =  14 mp =  14 lp =  8
out:    L 

in:     T
Lpos=8, Mpos=14, Rpos=15
rp =  15 mp =  14 lp =  8
out:    L 

in:     I
Lpos=8, Mpos=14, Rpos=16
rp =  16 mp =  14 lp =  8
out:    T 

in:     F
Lpos=8, Mpos=14, Rpos=17
rp

'NKNMWGEDRLLLTROMZYLWPHVZJQGFRCTTTJJADQXCOVNJJBBLIRZYWITPAZUTGNDDBBUVDQBGOZDMVACHRQMT'

In [22]:
igged = eg1.enigmatise(word, startset=opcode)
igged

in:     O
Lpos=8, Mpos=13, Rpos=4
rp =  4 mp =  13 lp =  8
out:    N 

in:     H
--- middle rotor step ---
Lpos=8, Mpos=14, Rpos=5
rp =  5 mp =  14 lp =  8
out:    K 

in:     W
Lpos=8, Mpos=14, Rpos=6
rp =  6 mp =  14 lp =  8
out:    N 

in:     H
Lpos=8, Mpos=14, Rpos=7
rp =  7 mp =  14 lp =  8
out:    M 

in:     A
Lpos=8, Mpos=14, Rpos=8
rp =  8 mp =  14 lp =  8
out:    W 

in:     T
Lpos=8, Mpos=14, Rpos=9
rp =  9 mp =  14 lp =  8
out:    G 

in:     A
Lpos=8, Mpos=14, Rpos=10
rp =  10 mp =  14 lp =  8
out:    E 

in:     B
Lpos=8, Mpos=14, Rpos=11
rp =  11 mp =  14 lp =  8
out:    D 

in:     E
Lpos=8, Mpos=14, Rpos=12
rp =  12 mp =  14 lp =  8
out:    R 

in:     A
Lpos=8, Mpos=14, Rpos=13
rp =  13 mp =  14 lp =  8
out:    L 

in:     U
Lpos=8, Mpos=14, Rpos=14
rp =  14 mp =  14 lp =  8
out:    L 

in:     T
Lpos=8, Mpos=14, Rpos=15
rp =  15 mp =  14 lp =  8
out:    L 

in:     I
Lpos=8, Mpos=14, Rpos=16
rp =  16 mp =  14 lp =  8
out:    T 

in:     F
Lpos=8, Mpos=14, Rpos=17
rp

'NKNMWGEDRLLLTROMZYLWPHVZJQGFRCTTTJJADQXCOVNJJBBLIRZYWITPAZUTGNDDBBUVDQBGOZDMVACHRQMT'

## Have now set up function 'enigmatise' to encode a message in enigma format
### Next step is to work on decoding it!

In [23]:
igged

'NKNMWGEDRLLLTROMZYLWPHVZJQGFRCTTTJJADQXCOVNJJBBLIRZYWITPAZUTGNDDBBUVDQBGOZDMVACHRQMT'

In [25]:
reword = eg1.enigmatise(igged, startset=opcode)
reword

in:     N
Lpos=8, Mpos=13, Rpos=4
rp =  4 mp =  13 lp =  8
out:    O 

in:     K
--- middle rotor step ---
Lpos=8, Mpos=14, Rpos=5
rp =  5 mp =  14 lp =  8
out:    H 

in:     N
Lpos=8, Mpos=14, Rpos=6
rp =  6 mp =  14 lp =  8
out:    W 

in:     M
Lpos=8, Mpos=14, Rpos=7
rp =  7 mp =  14 lp =  8
out:    H 

in:     W
Lpos=8, Mpos=14, Rpos=8
rp =  8 mp =  14 lp =  8
out:    A 

in:     G
Lpos=8, Mpos=14, Rpos=9
rp =  9 mp =  14 lp =  8
out:    T 

in:     E
Lpos=8, Mpos=14, Rpos=10
rp =  10 mp =  14 lp =  8
out:    A 

in:     D
Lpos=8, Mpos=14, Rpos=11
rp =  11 mp =  14 lp =  8
out:    B 

in:     R
Lpos=8, Mpos=14, Rpos=12
rp =  12 mp =  14 lp =  8
out:    E 

in:     L
Lpos=8, Mpos=14, Rpos=13
rp =  13 mp =  14 lp =  8
out:    A 

in:     L
Lpos=8, Mpos=14, Rpos=14
rp =  14 mp =  14 lp =  8
out:    U 

in:     L
Lpos=8, Mpos=14, Rpos=15
rp =  15 mp =  14 lp =  8
out:    T 

in:     T
Lpos=8, Mpos=14, Rpos=16
rp =  16 mp =  14 lp =  8
out:    I 

in:     R
Lpos=8, Mpos=14, Rpos=17
rp

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [26]:
print(igged)
print(reword)

NKNMWGEDRLLLTROMZYLWPHVZJQGFRCTTTJJADQXCOVNJJBBLIRZYWITPAZUTGNDDBBUVDQBGOZDMVACHRQMT
OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY


In [27]:
crib = 'OHWHATABEAUTIFULDAY'
lncrib = len(crib)

In [28]:
reword[23:23+lncrib]

'OHWHATABEAUTIFULDAY'

In [29]:
igged[:len(igged)-lncrib]

'NKNMWGEDRLLLTROMZYLWPHVZJQGFRCTTTJJADQXCOVNJJBBLIRZYWITPAZUTGNDDB'

#### Have chosen a crib (how to do this automatically will be a bigger problem later on)
#### first step is to find positions in the encoded text ('igged') that could feasibly contain the crib words, as defined by not encoding a letter as itself

In [30]:

status_of_crib_indexes = {k:True for k in range(len(igged))}
# print(status_of_crib_indexes)
for num, letter in enumerate(igged[:len(igged)-lncrib]):
    trange = range(num,num+lncrib)
    tcrib = igged[num:num+lncrib]
    for c, t in zip(crib, tcrib):
        if c == t:
            status_of_crib_indexes[num] = False
    
        
print(status_of_crib_indexes)
maybes = [k for k,v in status_of_crib_indexes.items() if v == True]
print(maybes)

{0: True, 1: False, 2: False, 3: False, 4: True, 5: True, 6: True, 7: False, 8: True, 9: True, 10: True, 11: True, 12: True, 13: True, 14: False, 15: True, 16: True, 17: False, 18: False, 19: False, 20: False, 21: False, 22: True, 23: True, 24: True, 25: False, 26: False, 27: False, 28: True, 29: False, 30: True, 31: False, 32: False, 33: False, 34: True, 35: True, 36: False, 37: True, 38: False, 39: False, 40: False, 41: False, 42: True, 43: False, 44: False, 45: True, 46: False, 47: False, 48: False, 49: False, 50: False, 51: True, 52: False, 53: True, 54: False, 55: True, 56: False, 57: False, 58: False, 59: True, 60: False, 61: True, 62: True, 63: False, 64: True, 65: True, 66: True, 67: True, 68: True, 69: True, 70: True, 71: True, 72: True, 73: True, 74: True, 75: True, 76: True, 77: True, 78: True, 79: True, 80: True, 81: True, 82: True, 83: True}
[0, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 22, 23, 24, 28, 30, 34, 35, 37, 42, 45, 51, 53, 55, 59, 61, 62, 64, 65, 66, 67, 68, 69, 70

#### Looks like there are many potential positions. This will be a problem later in the piece when trying to do more unknown messages. Will test out a method for a crib that I know is in the correct position 

In [32]:
mcrib = igged[23:23+lncrib]
print(crib)
print(mcrib)

OHWHATABEAUTIFULDAY
ZJQGFRCTTTJJADQXCOV


In [33]:
pairs = {i:{c,m} for i,c,m in zip(range(23+lncrib), crib, mcrib)}
pairs

{0: {'O', 'Z'},
 1: {'H', 'J'},
 2: {'Q', 'W'},
 3: {'G', 'H'},
 4: {'A', 'F'},
 5: {'R', 'T'},
 6: {'A', 'C'},
 7: {'B', 'T'},
 8: {'E', 'T'},
 9: {'A', 'T'},
 10: {'J', 'U'},
 11: {'J', 'T'},
 12: {'A', 'I'},
 13: {'D', 'F'},
 14: {'Q', 'U'},
 15: {'L', 'X'},
 16: {'C', 'D'},
 17: {'A', 'O'},
 18: {'V', 'Y'}}

In [34]:
links = {character:len([pair for pair in pairs.values() if character in pair]) for character in entry}   
print(links)
hilinks = {k:v for k,v in links.items() if v > 0}
hilinks

{'A': 5, 'B': 1, 'C': 2, 'D': 2, 'E': 1, 'F': 2, 'G': 1, 'H': 2, 'I': 1, 'J': 3, 'K': 0, 'L': 1, 'M': 0, 'N': 0, 'O': 2, 'P': 0, 'Q': 2, 'R': 1, 'S': 0, 'T': 5, 'U': 2, 'V': 1, 'W': 1, 'X': 1, 'Y': 1, 'Z': 1}


{'A': 5,
 'B': 1,
 'C': 2,
 'D': 2,
 'E': 1,
 'F': 2,
 'G': 1,
 'H': 2,
 'I': 1,
 'J': 3,
 'L': 1,
 'O': 2,
 'Q': 2,
 'R': 1,
 'T': 5,
 'U': 2,
 'V': 1,
 'W': 1,
 'X': 1,
 'Y': 1,
 'Z': 1}

#### 'links' is just record for each letter of the alphabet, how many times it appears paired to a cypher or natural letter in either of the encoded or raw messages
#### 'hilinks' is just those letters that have more than 2 pairs (links)
#### 'hipairs' is now preparing, for each of the 'hilinks' letters of interest, a dictionary record of which other letters they are paired to, and in what position

In [35]:
mostlinks = sorted(hilinks.values(),reverse=True)[0]
mostlinks

5

In [36]:
best_characters = [k for k,v in hilinks.items() if v == mostlinks]
best_characters

['A', 'T']

In [37]:
hipairs = {}
for character in hilinks.keys():
    hset = {k:list(pair) for k,pair in pairs.items() if character in pair}
    newresult = {}
    for k,v in hset.items():
        v.remove(character)
        if v[0] in hilinks.keys():
            newresult[k] = v[0]
#     print('hset:      ',hset)
#     print('newresult: ',newresult)
    if len(newresult) > 0:
        hipairs[character] = newresult
#     hipairs[character] = {k:pair.remove(character) for k,pair in pairs.items() if character in pair}
## maybe try a comprehension again later, involves a few steps...

    
hipairs

{'A': {4: 'F', 6: 'C', 9: 'T', 12: 'I', 17: 'O'},
 'B': {7: 'T'},
 'C': {6: 'A', 16: 'D'},
 'D': {13: 'F', 16: 'C'},
 'E': {8: 'T'},
 'F': {4: 'A', 13: 'D'},
 'G': {3: 'H'},
 'H': {1: 'J', 3: 'G'},
 'I': {12: 'A'},
 'J': {1: 'H', 10: 'U', 11: 'T'},
 'L': {15: 'X'},
 'O': {0: 'Z', 17: 'A'},
 'Q': {2: 'W', 14: 'U'},
 'R': {5: 'T'},
 'T': {5: 'R', 7: 'B', 8: 'E', 9: 'A', 11: 'J'},
 'U': {10: 'J', 14: 'Q'},
 'V': {18: 'Y'},
 'W': {2: 'Q'},
 'X': {15: 'L'},
 'Y': {18: 'V'},
 'Z': {0: 'O'}}

#### for now just doing this entire process for one letter, 'a', set as variable 'x'. Will need to put this whole process into a function later so it can iterate through each of the letters - I think just the letters in hilinks, as opposed to every letter in the alphabet
Though thinking further - maybe it is alright to just start with the letter with the most connections? The chain space for a given cypher/crib set of characters will be the same, the starting letter is just to give the function somewhere to start?

In [38]:
def make_connections(x, dx, loops={}, deadends={}, itr = 1):
    """for sorting through a hipairs dictionary of letters of interest and their corresponding paired letters. 
    Used with a WHILE loop, can recursively search through 'chains' or paths that a letter sequence can take 
    by following pairs from one letter to the next. Looks for 'loops', where a chain path can return to its original
    starting letter. Records other chains as deadends
    """
    working_dict = deepcopy(dx)
    # print(nd)
    for iD,chain in dx.items():
        current_end = chain[-1]
        cx = hipairs[current_end]
#         print(cx)
        for jid,conxn in enumerate(cx.values()):
            key = round(iD+jid/10**itr,5)

            if conxn != current_end:
                working_dict[key] = dx[iD] + conxn
    #     print(cx)
    print('all connections')
    pprint(working_dict)

    dx={}
    for kid,chain in working_dict.items():
        if chain[-1] == chain[-3]:# and len(v):
            chain = chain[:-1]
            deadends[kid] = chain
        elif chain[-1] == x and len(chain) > 3:   ## ie we're legit back to the start after a loop
            loops[kid] = chain
        else:
            dx[kid] = chain
    print('useful connections: ')
    pprint(dx)
    print('loops:')
    pprint(loops)
    return dx, loops, deadends

In [39]:
def find_loops(starting_character='A'):
    working_dict = {i+0.0:starting_character for i in range(len(hipairs[starting_character]))}
    for i,v in zip(range(len(hipairs[starting_character])), hipairs[starting_character].values()):
        working_dict[i] += v
#     pprint(ax)

    run = 1
    found_loops = {}
    dead_ends = {}
    while len(working_dict) > 0:
        print(f"run {run}")
        working_dict, found_loops, dead_ends = make_connections(starting_character, working_dict,found_loops,dead_ends,run)
        run +=1
    
    return found_loops, dead_ends

In [40]:
aloops, d_ends = find_loops(best_characters[0])

run 1
all connections
{0.0: 'AFA',
 0.1: 'AFD',
 1.0: 'ACA',
 1.1: 'ACD',
 2.0: 'ATR',
 2.1: 'ATB',
 2.2: 'ATE',
 2.3: 'ATA',
 2.4: 'ATJ',
 3.0: 'AIA',
 4.0: 'AOZ',
 4.1: 'AOA'}
useful connections: 
{0.1: 'AFD',
 1.1: 'ACD',
 2.0: 'ATR',
 2.1: 'ATB',
 2.2: 'ATE',
 2.4: 'ATJ',
 4.0: 'AOZ'}
loops:
{}
run 2
all connections
{0.1: 'AFDF',
 0.11: 'AFDC',
 1.1: 'ACDF',
 1.11: 'ACDC',
 2.0: 'ATRT',
 2.1: 'ATBT',
 2.2: 'ATET',
 2.4: 'ATJH',
 2.41: 'ATJU',
 2.42: 'ATJT',
 4.0: 'AOZO'}
useful connections: 
{0.11: 'AFDC', 1.1: 'ACDF', 2.4: 'ATJH', 2.41: 'ATJU'}
loops:
{}
run 3
all connections
{0.11: 'AFDCA',
 0.111: 'AFDCD',
 1.1: 'ACDFA',
 1.101: 'ACDFD',
 2.4: 'ATJHJ',
 2.401: 'ATJHG',
 2.41: 'ATJUJ',
 2.411: 'ATJUQ'}
useful connections: 
{2.401: 'ATJHG', 2.411: 'ATJUQ'}
loops:
{0.11: 'AFDCA', 1.1: 'ACDFA'}
run 4
all connections
{2.401: 'ATJHGH', 2.411: 'ATJUQW', 2.4111: 'ATJUQU'}
useful connections: 
{2.411: 'ATJUQW'}
loops:
{0.11: 'AFDCA', 1.1: 'ACDFA'}
run 5
all connections
{2.411: 'ATJUQWQ'}

In [41]:
aloops

{1.1: 'ACDFA', 0.11: 'AFDCA'}

In [42]:
d_ends

{0.0: 'AF',
 1.0: 'AC',
 3.0: 'AI',
 2.3: 'AT',
 4.1: 'AO',
 2.0: 'ATR',
 4.0: 'AOZ',
 0.1: 'AFD',
 2.1: 'ATB',
 2.2: 'ATE',
 1.11: 'ACD',
 2.42: 'ATJ',
 2.4: 'ATJH',
 2.41: 'ATJU',
 1.101: 'ACDF',
 0.111: 'AFDC',
 2.401: 'ATJHG',
 2.4111: 'ATJUQ',
 2.411: 'ATJUQW'}

In [43]:
def rationalise_loops(indict):
    invals = list(set(indict.values()))
#     invals = list(set(invals))
    for i,loop in enumerate(invals):
        test = deepcopy(invals)
#         print(test)
        test[i] = loop[::-1]
        test = list(set(test))
#         print(test)
        if len(test) != len(invals):
            invals[i] = loop[::-1]
#         print(invals)
    invals = list(set(invals))

    return invals

In [44]:
rdends = rationalise_loops(d_ends)
print(rdends)

['ATR', 'ATB', 'ATJH', 'AOZ', 'ATJU', 'ACDF', 'ACD', 'AFD', 'AFDC', 'AO', 'AC', 'ATJUQW', 'AT', 'AF', 'ATE', 'ATJUQ', 'AI', 'ATJ', 'ATJHG']


In [45]:
def unsub_list(inlist):
    inlist.sort()
    unique = []
    for i in range(len(inlist)-1):
        chain = inlist[i]
        next_chain = inlist[i+1]
#         print(chain, next_chain, chain in next_chain)
        if chain not in next_chain:
            unique.append(chain)

    unique.append(inlist[-1])

    return unique

In [46]:
uqends = unsub_list(rdends)
uqends

['ACDF', 'AFDC', 'AI', 'AOZ', 'ATB', 'ATE', 'ATJHG', 'ATJUQW', 'ATR']

In [47]:
aloops

{1.1: 'ACDFA', 0.11: 'AFDCA'}

In [48]:
rloops = rationalise_loops(aloops)
rloops

['AFDCA']

['AYOULBWA']

In [49]:
lcheck = set(aloops.values())
print(lcheck)
fuqends = deepcopy(uqends)
for uchain in uqends:
#     print(uchain)
    for l in lcheck:
        print (uchain, l, uchain in l)
        if uchain in l:
            fuqends.remove(uchain)
            
fuqends

{'ACDFA', 'AFDCA'}
ACDF ACDFA True
ACDF AFDCA False
AFDC ACDFA False
AFDC AFDCA True
AI ACDFA False
AI AFDCA False
AOZ ACDFA False
AOZ AFDCA False
ATB ACDFA False
ATB AFDCA False
ATE ACDFA False
ATE AFDCA False
ATJHG ACDFA False
ATJHG AFDCA False
ATJUQW ACDFA False
ATJUQW AFDCA False
ATR ACDFA False
ATR AFDCA False


['AI', 'AOZ', 'ATB', 'ATE', 'ATJHG', 'ATJUQW', 'ATR']

['ACHI', 'AK']

In [50]:
hipairs

{'A': {4: 'F', 6: 'C', 9: 'T', 12: 'I', 17: 'O'},
 'B': {7: 'T'},
 'C': {6: 'A', 16: 'D'},
 'D': {13: 'F', 16: 'C'},
 'E': {8: 'T'},
 'F': {4: 'A', 13: 'D'},
 'G': {3: 'H'},
 'H': {1: 'J', 3: 'G'},
 'I': {12: 'A'},
 'J': {1: 'H', 10: 'U', 11: 'T'},
 'L': {15: 'X'},
 'O': {0: 'Z', 17: 'A'},
 'Q': {2: 'W', 14: 'U'},
 'R': {5: 'T'},
 'T': {5: 'R', 7: 'B', 8: 'E', 9: 'A', 11: 'J'},
 'U': {10: 'J', 14: 'Q'},
 'V': {18: 'Y'},
 'W': {2: 'Q'},
 'X': {15: 'L'},
 'Y': {18: 'V'},
 'Z': {0: 'O'}}

In [51]:
rloops

['AFDCA']

In [52]:
mainloop = rloops[0]
menu= {}
for i,char in enumerate(mainloop[:-1]):
    next_char = mainloop[i+1]
    wdict = hipairs[char]
#     print(wdict)
    position = [k for k,v in wdict.items() if v == next_char][0]
    ### note that I'm just picking the first one where there are double (or more) linkages
    ### not sure if this matters for now or if its better to somehow include both linkages in the menu
    ### revisit later depending on bombe methodology
#     print(position)
    print(i,char,next_char,position)
    menu[position] = {'in':char, 'out':next_char, 'menu_link':position}
menu

0 A F 4
1 F D 13
2 D C 16
3 C A 6


{4: {'in': 'A', 'out': 'F', 'menu_link': 4},
 13: {'in': 'F', 'out': 'D', 'menu_link': 13},
 16: {'in': 'D', 'out': 'C', 'menu_link': 16},
 6: {'in': 'C', 'out': 'A', 'menu_link': 6}}

In [53]:
len(menu)

4

In [54]:
for ends in fuqends:
    current_len = len(menu)
    print(current_len)
    for i,char in enumerate(ends[:-1]):
        next_char = ends[i+1]
#         print(i,char,next_char)
        wdict = hipairs[char]
        position = [k for k,v in wdict.items() if v == next_char][0]
        menu[position] = {'in':char, 'out':next_char, 'menu_link':position}
        
menu

4
5
7
9
10
13
16


{4: {'in': 'A', 'out': 'F', 'menu_link': 4},
 13: {'in': 'F', 'out': 'D', 'menu_link': 13},
 16: {'in': 'D', 'out': 'C', 'menu_link': 16},
 6: {'in': 'C', 'out': 'A', 'menu_link': 6},
 12: {'in': 'A', 'out': 'I', 'menu_link': 12},
 17: {'in': 'A', 'out': 'O', 'menu_link': 17},
 0: {'in': 'O', 'out': 'Z', 'menu_link': 0},
 9: {'in': 'A', 'out': 'T', 'menu_link': 9},
 7: {'in': 'T', 'out': 'B', 'menu_link': 7},
 8: {'in': 'T', 'out': 'E', 'menu_link': 8},
 11: {'in': 'T', 'out': 'J', 'menu_link': 11},
 1: {'in': 'J', 'out': 'H', 'menu_link': 1},
 3: {'in': 'H', 'out': 'G', 'menu_link': 3},
 10: {'in': 'J', 'out': 'U', 'menu_link': 10},
 14: {'in': 'U', 'out': 'Q', 'menu_link': 14},
 2: {'in': 'Q', 'out': 'W', 'menu_link': 2},
 5: {'in': 'T', 'out': 'R', 'menu_link': 5}}

In [148]:
menu

{9: {'in': 'A', 'out': 'W', 'menu_link': 9},
 2: {'in': 'W', 'out': 'B', 'menu_link': 2},
 7: {'in': 'B', 'out': 'L', 'menu_link': 7},
 14: {'in': 'L', 'out': 'U', 'menu_link': 14},
 10: {'in': 'U', 'out': 'O', 'menu_link': 10},
 0: {'in': 'O', 'out': 'Y', 'menu_link': 0},
 6: {'in': 'Y', 'out': 'A', 'menu_link': 6},
 17: {'in': 'A', 'out': 'C', 'menu_link': 17},
 1: {'in': 'C', 'out': 'H', 'menu_link': 1},
 3: {'in': 'H', 'out': 'I', 'menu_link': 3},
 4: {'in': 'A', 'out': 'K', 'menu_link': 4},
 5: {'in': 'T', 'out': 'D', 'menu_link': 5}}

In [149]:
for k,m in menu.items():
    l = m['menu_link']
    l = entry[l-1]
    l = 'ZZ' + l
#     print(l)
    menu[k]['menu_link'] = l
    menu[k]['conx_in'] = {}
    menu[k]['conx_out'] = {}
menu

{9: {'in': 'A', 'out': 'W', 'menu_link': 'ZZI', 'conx_in': {}, 'conx_out': {}},
 2: {'in': 'W', 'out': 'B', 'menu_link': 'ZZB', 'conx_in': {}, 'conx_out': {}},
 7: {'in': 'B', 'out': 'L', 'menu_link': 'ZZG', 'conx_in': {}, 'conx_out': {}},
 14: {'in': 'L',
  'out': 'U',
  'menu_link': 'ZZN',
  'conx_in': {},
  'conx_out': {}},
 10: {'in': 'U',
  'out': 'O',
  'menu_link': 'ZZJ',
  'conx_in': {},
  'conx_out': {}},
 0: {'in': 'O', 'out': 'Y', 'menu_link': 'ZZZ', 'conx_in': {}, 'conx_out': {}},
 6: {'in': 'Y', 'out': 'A', 'menu_link': 'ZZF', 'conx_in': {}, 'conx_out': {}},
 17: {'in': 'A',
  'out': 'C',
  'menu_link': 'ZZQ',
  'conx_in': {},
  'conx_out': {}},
 1: {'in': 'C', 'out': 'H', 'menu_link': 'ZZA', 'conx_in': {}, 'conx_out': {}},
 3: {'in': 'H', 'out': 'I', 'menu_link': 'ZZC', 'conx_in': {}, 'conx_out': {}},
 4: {'in': 'A', 'out': 'K', 'menu_link': 'ZZD', 'conx_in': {}, 'conx_out': {}},
 5: {'in': 'T', 'out': 'D', 'menu_link': 'ZZE', 'conx_in': {}, 'conx_out': {}}}

In [142]:
len(menu)

12

In [151]:
for pos,mdict in menu.items():
    sin = mdict['in']
    for k,v in menu.items():
        if k == pos:
            pass
        elif v['in'] == sin:
            menu[pos]['conx_in'][k] = 'in'
        elif v['out'] == sin:
            menu[pos]['conx_in'][k] = 'out'
    
    sout = mdict['out']
    for k,v in menu.items():
        if k == pos:
            pass
        elif v['in'] == sout:
            menu[pos]['conx_out'][k] = 'in'
        elif v['out'] == sout:
            menu[pos]['conx_out'][k] = 'out'
menu

{9: {'in': 'A',
  'out': 'W',
  'menu_link': 'ZZI',
  'conx_in': {6: 'out', 17: 'in', 4: 'in'},
  'conx_out': {2: 'in'}},
 2: {'in': 'W',
  'out': 'B',
  'menu_link': 'ZZB',
  'conx_in': {9: 'out'},
  'conx_out': {7: 'in'}},
 7: {'in': 'B',
  'out': 'L',
  'menu_link': 'ZZG',
  'conx_in': {2: 'out'},
  'conx_out': {14: 'in'}},
 14: {'in': 'L',
  'out': 'U',
  'menu_link': 'ZZN',
  'conx_in': {7: 'out'},
  'conx_out': {10: 'in'}},
 10: {'in': 'U',
  'out': 'O',
  'menu_link': 'ZZJ',
  'conx_in': {14: 'out'},
  'conx_out': {0: 'in'}},
 0: {'in': 'O',
  'out': 'Y',
  'menu_link': 'ZZZ',
  'conx_in': {10: 'out'},
  'conx_out': {6: 'in'}},
 6: {'in': 'Y',
  'out': 'A',
  'menu_link': 'ZZF',
  'conx_in': {0: 'out'},
  'conx_out': {9: 'in', 17: 'in', 4: 'in'}},
 17: {'in': 'A',
  'out': 'C',
  'menu_link': 'ZZQ',
  'conx_in': {9: 'in', 6: 'out', 4: 'in'},
  'conx_out': {1: 'in'}},
 1: {'in': 'C',
  'out': 'H',
  'menu_link': 'ZZA',
  'conx_in': {17: 'out'},
  'conx_out': {3: 'in'}},
 3: {'in'

In [155]:
with open('menu_transfer.pickle', 'wb') as outfile:
    dill.dump(menu,outfile)

In [128]:
hipairs

{'A': {4: 'K', 6: 'Y', 9: 'W', 17: 'C'},
 'B': {2: 'W', 7: 'L', 15: 'L'},
 'C': {1: 'H', 17: 'A'},
 'D': {5: 'T', 16: 'Q'},
 'E': {8: 'K'},
 'F': {13: 'T'},
 'H': {1: 'C', 3: 'I'},
 'I': {3: 'H', 12: 'R'},
 'K': {4: 'A', 8: 'E'},
 'L': {7: 'B', 14: 'U', 15: 'B'},
 'O': {0: 'Y', 10: 'U'},
 'P': {18: 'Y'},
 'Q': {16: 'D'},
 'R': {12: 'I'},
 'T': {5: 'D', 11: 'V', 13: 'F'},
 'U': {10: 'O', 14: 'L'},
 'V': {11: 'T'},
 'W': {2: 'B', 9: 'A'},
 'Y': {0: 'O', 6: 'A', 18: 'P'}}